<a href="https://colab.research.google.com/github/chw8207/pytorch_study/blob/main/softmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn.functional as F

In [2]:
torch.manual_seed(1)

###  파이토치로 소프트맥스의 비용 함수 구현하기 (로우-레벨)

In [3]:
z = torch.FloatTensor([1,2,3])

In [4]:
# 텐서를 소프트맥스 함수 입력으로 사용하고 확인해보기
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [5]:
# 원소들의 값의 합이 1인지 확인해보기
hypothesis.sum()

tensor(1.)